In [1]:
# public packages
import andes
import ams
import cvxpy as cp
import numpy as np
import gurobipy

# local packages
import anci_fun as af

# v1 try to include dynamic constraints using reduced input/output NN model

# ------ nn information ------
# input:   [M, D]
# output:  [rocof_max, fnadir, dtheta_max, busV_min, eig_max, eig_max_]
# nn_diam: [27, 512, 6]

# Para preparetion

In [2]:
# neural network parameters 
nn_name = 'trained_model.pth'
nn_weights, nn_bias, nn_diam = af.extract_nn_para(nn_name) # three list

# nn_diam = [27, 512, 6]

In [3]:
# MD parameters
vsg_num = 8

lb_M = np.array([0, 0, 0, 0, 0, 0, 0, 0])
ub_M = np.array([6, 6, 6, 6, 6, 6, 6, 6])

lb_D = np.array([0, 0, 0, 0, 0, 0, 0, 0])
ub_D = np.array([4, 4, 4, 4, 4, 4, 4, 4])

In [4]:
sf_load = 1.0

# y = [rocof_max, fnadir, dtheta_max, busV_min, eig_max, eig_max_]
lb_y = np.array([-0.5, -0.6, -3.14, 0, -100, -100])
ub_y = np.array([ 0.5,  0.6,  3.14, 2,    0,    0])

# Load ams model

In [5]:
ssp = ams.load('ams_case123_REGCV1_v5.xlsx')

ssp.LDOPF.run()

ssp.LDOPF.om.mdl.solve()

ssp.LDOPF.om.vars

LDOPF solved as optimal in 0.0968 seconds with exit code 0.


OrderedDict([('pg', Variable((11,))),
             ('pn', Variable((117,))),
             ('qg', Variable((11,))),
             ('qn', Variable((117,))),
             ('vsq', Variable((117,))),
             ('pl', Variable((116,))),
             ('ql', Variable((116,)))])

# Define new variables, constraints, and obj

In [6]:
# M and D

M = cp.Variable(vsg_num, name="M")
D = cp.Variable(vsg_num, name="D")

constraints_M = [M >= lb_M, M <= ub_M]
constraints_D = [D >= lb_D, D <= ub_D]


In [7]:
# data for the neural network linearization
pg    = ssp.LDOPF.om.pg

x1    = cp.hstack([pg, M, D])  # this version doesn't incelu load scaling factor

w1    = nn_weights[0]
w2    = nn_weights[1]

b1    = nn_bias[0]
b2    = nn_bias[1]

hdown = -100
hup   = 100

In [8]:
# neural network linization

z = cp.Variable(nn_diam[1], name="z")
a = cp.Variable(nn_diam[1], name="a", boolean=True)

z_ = w1 @ x1 + b1

y = w2 @ z + b2

constraints_nn = [
                    z <= z_ - hdown * (1 - a),
                    z >= z_,
                    z <= hup * a,
                    z >= 0,
                    # output constraints
                    y[0] <= ub_y[0],       # RoCoF
                    y[0] >= lb_y[0],
                    # y[1] <= ub_y[1],       # fnadir
                    y[1] >= lb_y[1],
                    # y[2] <= ub_y[2],       # dtheta_max
                    # y[2] >= lb_y[2],
                    # y[3] <= ub_y[3],       # eig_max
                    # y[3] >= lb_y[3]      
                ]

In [9]:
constriants_ams = [ssp.LDOPF.om.constrs[cname] for cname in ssp.LDOPF.om.constrs.keys()]

In [10]:
constraints = constraints_M + constraints_D + constraints_nn + constriants_ams

# constraints = constriants_ams + constraints_M + constraints_D

# Formulate new problem

pg = ['PV_1',
 'PV_10',
 'PV_11',
 'PV_2',
 'PV_3',
 'PV_4',
 'PV_5',
 'PV_6',
 'PV_7',
 'PV_8',
 'PV_9']

 SG: PV_1, PV_2, PV_3 (slack bus)

 IBR: PV_4, PV_5, ..., PV_11

In [11]:
c2 = np.array([0.01, 0, 0, 0.01, 0.02, 0, 0, 0, 0, 0, 0])
c1 = np.array([  20, 0, 0,   20,   30, 0, 0, 0, 0, 0, 0])

d  = np.array([0, 0, 0, 0.001, 0, 0, 0.001, 0])

pg_power = pg**2

cost = cp.sum(c2 @ pg_power + c1 @ pg + d @ M)

obj = cp.Minimize(cost)

In [12]:
# Create the optimization problem
prob = cp.Problem(obj, constraints)

# Solve the optimization problem

# prob.solve(solver=cp.GUROBI, verbose=True)

prob.solve(solver=cp.ECOS_BB, verbose=True)

# prob.solve()

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Sep 12 10:52:03 PM: Your problem has 1645 variables, 24 constraints, and 0 parameters.
(CVXPY) Sep 12 10:52:03 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Sep 12 10:52:03 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Sep 12 10:52:03 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Sep 12 10:52:03 PM: Compiling problem (target solver=ECOS_BB).
(CVXPY) Sep 12 10:52:03 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuff


ECOS 2.0.10 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +1.487e+01  -1.100e+05  +1e+05  5e-02  7e-01  1e+00  4e+01    ---    ---    1  1  - |  -  - 
 1  -5.989e+01  -5.249e+04  +7e+04  2e-02  3e-01  4e+00  2e+01  0.6375  1e-01   1  1  1 |  0  0
 2  -2.294e+01  -3.764e+04  +4e+04  2e-02  2e-01  6e+00  1e+01  0.4966  3e-01   1  1  1 |  0  0
 3  +1.184e+00  -1.015e+04  +1e+04  5e-03  3e-02  4e+00  3e+00  0.8927  2e-01   1  1  1 |  0  0
 4  +1.562e+00  -2.928e+03  +3e+03  1e-03  8e-03  1e+00  9e-01  0.7410  5e-02   1  1  1 |  0  0
 5  +1.881e+00  -1.987e+03  +2e+03  9e-04  5e-03  8e-01  6e-01  0.3838  1e-01   1  1  1 |  0  0
 6  +2.713e+00  -2.405e+03  +2e+03  1e-03  4e-03  1e+00  5e-01  0.8294  8e-01   1  1  1 |  0  0
 7  +2.876e+00  -2.389e+03  +2e+03  1e-03  4e-03  2e+00  5e-01  0.1992  9e-01   1  1  1 |  0  0
 8  +2.902e+00  -6.034e+02  +5e+02  3e-04  1e

In [ ]:
ssp.LDOPF.om.pg.value

array([0.02      , 0.02180299, 0.02727535, 0.02      , 0.04      ,
       0.06460351, 0.07714397, 0.01763018, 0.02636903, 0.02555273,
       0.01262225])

In [ ]:
M.value

array([0.35426634, 0.32230359, 3.08734316, 0.        , 0.69477843,
       0.62209887, 0.        , 5.98141608])

In [ ]:
D.value

array([1.91649179, 2.19009182, 0.59658497, 1.81851059, 0.76965922,
       0.18199612, 0.85678189, 0.76842172])

# 